In [1]:
import pandas as pd
import os

# GF TODO: check if excel present, otherwise download it
# wget http://archive.ics.uci.edu/ml/machine-learning-databases/00502/online_retail_II.xlsx

excel_with_path  = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.xlsx'
pickle_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.pkl'
csv_with_path = '/eos/user/f/franzoni/SWAN_projects/proton/online_retail_II.csv'

df = None

if os.path.isfile(pickle_with_path):
    print('-> picke already exists, much faster\n')
    df = pd.read_pickle(pickle_with_path)
else:
    print('-> picke does not exist, go to excel, and create it\n')
    df1 = pd.read_excel (excel_with_path,'Year 2009-2010')
    df2 = pd.read_excel (excel_with_path,'Year 2010-2011')
    df = pd.concat([df1, df2])
    df.to_pickle(pickle_with_path)

df['Transaction'] = df.Quantity * df.Price
df=df.rename(columns={"Customer ID": "Customer_ID"})

# df.to_csv(csv_with_path, encoding = 'utf-8')

# GF TODO: there are nan 
# GF TODO: there are fields which are 'empty char', e.g. in the customer_id

-> picke already exists, much faster



In [2]:
df.head(5)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,Transaction
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


## What do I want to extract from this dataset?
- https://en.wikipedia.org/wiki/Exploratory_data_analysis
- how many customers
- how many customer in which country
- how much do customers spend => make a PLOT!
- trends of spending: overall, by the largest customer, by the smaller customers

- how many different types of items
- what kind of items are bought the most (by NUMBER of by REVENUE), are cancelled the most, 
- how much revenue per type of item bought
- correlation between 

In [3]:
print('There are %d unique customers'%len(df.Customer_ID.unique()))

There are 5943 unique customers


In [4]:
print('Customers are from %d countries'%len(df.Country.unique()))

Customers are from 43 countries


In [5]:
# GF can be removed
# df.groupby('Country').Customer_ID.unique()

In [6]:
print('91 % of the customers are from the UK, 2% from Germany, 1.5% are from France. \
 The remaining 40 countries have less than 1% of the customers')

df.groupby('Country').Customer_ID.unique().agg([len])    \
    .rename(columns={"len": "num_customers"})            \
    .sort_values(by=['num_customers'],ascending=False)


91 % of the customers are from the UK, 2% from Germany, 1.5% are from France.  The remaining 40 countries have less than 1% of the customers


,num_customers
Country,
United Kingdom,5411
Germany,107
France,96
Spain,41
Belgium,29
Portugal,25
Switzerland,23
Netherlands,23
Sweden,20


In [22]:
# the largest customer by total net revenue (i.e. taking into account cancellations)
# is in the UK and has ID 18102.0
df.groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                  \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Total Customer Revenue'],ascending=False).head(10)        

,Total Customer Revenue,Minimum Transaction,Maximum Transaction,Number of Transaction
Customer_ID,,,,
18102.0,598215.22,-3580.80,3884.00,1068.0
14646.0,523342.07,-947.08,4992.00,3890.0
14156.0,296564.69,-2736.16,2736.16,4130.0
14911.0,270248.53,-1687.17,1687.17,11613.0
17450.0,233579.39,-3825.36,7144.72,448.0
13694.0,190825.52,-864.00,952.00,1558.0
17511.0,171885.98,-185.30,1401.60,2134.0
12415.0,143269.29,-425.00,1718.40,990.0
16684.0,141502.25,-816.00,1836.00,749.0


In [23]:
df.groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                  \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Number of Transaction'],ascending=False).head(10)    

,Total Customer Revenue,Minimum Transaction,Maximum Transaction,Number of Transaction
Customer_ID,,,,
17841.0,69516.19,-88.50,349.50,13097.0
14911.0,270248.53,-1687.17,1687.17,11613.0
12748.0,49970.13,-1829.84,850.50,7307.0
14606.0,30094.38,-55.10,139.30,6709.0
14096.0,57120.91,-1100.44,1599.26,5128.0
15311.0,113513.07,-275.00,408.00,4717.0
14156.0,296564.69,-2736.16,2736.16,4130.0
14646.0,523342.07,-947.08,4992.00,3890.0
13089.0,113214.19,-1051.20,1051.20,3438.0


In [36]:
# the customer with the largest number of transactions is 
# is in the UK and has ID 17841.0
df.loc[df.Transaction>0]                                                                                      \
    .groupby('Customer_ID').Transaction.agg([sum,min,max,len])                                                \
        .rename(columns={'sum': 'Total Customer Revenue',                                                     \
                         'min': 'Min Transaction','max': 'Max Transaction',                                   \
                            'len' : 'Number of Transaction'})                                                 \
              .sort_values(by=['Number of Transaction'],ascending=False).head(10)    

,Total Customer Revenue,Min Transaction,Max Transaction,Number of Transaction
Customer_ID,,,,
17841.0,70884.07,0.19,349.50,12890.0
14911.0,295972.63,1.45,1687.17,11245.0
12748.0,56599.39,0.06,850.50,7228.0
14606.0,30861.06,0.12,139.30,6566.0
14096.0,65164.79,0.42,1599.26,5111.0
15311.0,116771.16,0.21,408.00,4434.0
14156.0,313946.37,1.25,2736.16,4048.0
14646.0,528602.52,0.39,4992.00,3849.0
13089.0,116737.86,1.68,1051.20,3362.0


In [25]:
# the UK generates the largest revenue (16M over the two years),
# surprisingly EIRE is the second largest source of revenue, with 0.6M (despite having only 6 customers, they must be huge customers)
df.groupby('Country').Transaction.sum().sort_values(ascending=False)

Country
United Kingdom          1.638258e+07
EIRE                    6.155196e+05
Netherlands             5.485250e+05
Germany                 4.179886e+05
France                  3.281918e+05
Australia               1.671291e+05
Switzerland             9.972876e+04
Spain                   9.185948e+04
Sweden                  8.780942e+04
Denmark                 6.574109e+04
Belgium                 6.357449e+04
Portugal                5.313680e+04
Japan                   4.377658e+04
Channel Islands         4.145361e+04
Norway                  3.924303e+04
Italy                   3.067935e+04
Finland                 2.951445e+04
Cyprus                  2.416274e+04
Austria                 2.317760e+04
Greece                  1.899549e+04
Hong Kong               1.403930e+04
Singapore               1.315816e+04
Israel                  1.110722e+04
Poland                  1.052809e+04
United Arab Emirates    9.948650e+03
Unspecified             9.687320e+03
Lithuania               6.5537

In [9]:
df.groupby('Country').Transaction.sum().sort_values(ascending=False)

Country
United Kingdom          1.638258e+07
EIRE                    6.155196e+05
Netherlands             5.485250e+05
Germany                 4.179886e+05
France                  3.281918e+05
Australia               1.671291e+05
Switzerland             9.972876e+04
Spain                   9.185948e+04
Sweden                  8.780942e+04
Denmark                 6.574109e+04
Belgium                 6.357449e+04
Portugal                5.313680e+04
Japan                   4.377658e+04
Channel Islands         4.145361e+04
Norway                  3.924303e+04
Italy                   3.067935e+04
Finland                 2.951445e+04
Cyprus                  2.416274e+04
Austria                 2.317760e+04
Greece                  1.899549e+04
Hong Kong               1.403930e+04
Singapore               1.315816e+04
Israel                  1.110722e+04
Poland                  1.052809e+04
United Arab Emirates    9.948650e+03
Unspecified             9.687320e+03
Lithuania               6.5537